In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json

In [ ]:
template = """
아래 프롬프트를 서로 다른 모델에 실행한 결과인데, 두 결과를 비교해서 어떤 결과가 더 좋은지 판단해줘.
응답 형식은 1번 모델이 더 좋다면 '1', 2번 모델이 더 좋다면 '2'를 반환해줘.
해당 모델을 선택한 이유도 작성해줘.
결과는 json 형식으로 반환해.

<prompt>
{prompt}
</prompt>

<model1 결과>
{model1}
</model1>

<model2 결과>
{model2}
</model2>

<Output_Format>
{{"result": 1 or 2, "reason": "이유를 작성해주세요."}}
</Output_Format>
"""

llm = ChatOpenAI(model="gpt-4o-mini")
prompt =  PromptTemplate(
    template=template,
    input_variables=['prompt', 'model1', 'model2']
)
parser = JsonOutputParser()
chain = prompt | llm | parser

In [ ]:
# GPT 평가
# finetuned vs not finetuned
import asyncio
import json
import os
from tqdm.asyncio import tqdm  # 비동기 tqdm

# 저장된 데이터 경로
OUTPUT_FILE = "/Users/no.2/Desktop/GitHub/cpplab-sllm/data/eval_results/finetuned_notfinetuned_eval_results.json"
FAILED_REQUESTS_FILE = "/Users/no.2/Desktop/GitHub/cpplab-sllm/data/eval_results/finetuned_notfinetuned_eval_results.json"

from datasets import load_dataset
dataset = load_dataset("hyeongmin99/cpplab_sllm")

# 동시 작업 수 제한
semaphore = asyncio.Semaphore(10)
MAX_RETRIES = 3  # 최대 재시도 횟수

# 기존 저장된 데이터 불러오기
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
        try:
            results = json.load(f)
            print(f"✅ 기존 저장된 데이터 로드 성공 ({len(results)}개)")
        except json.JSONDecodeError:
            print("⚠️ 기존 데이터 로드 실패. 새로운 파일로 진행합니다.")
            results = [None] * len(dataset['test'])
else:
    results = [None] * len(dataset['test'])

# None인 데이터만 다시 실행할 리스트 생성
retry_indices = [i for i, result in enumerate(results) if result is None]
print(f"⚠️ {len(retry_indices)}개의 실패한 작업을 다시 시도합니다.")

# 실패한 요청 저장 리스트
failed_requests = []

with open('/Users/no.2/Desktop/GitHub/cpplab-sllm/data/inference_outputs/finetuned_outputs.json', "r", encoding="utf-8") as file:
    finetuened_outputs = json.load(file)
with open('/Users/no.2/Desktop/GitHub/cpplab-sllm/data/inference_outputs/notfinetuned_outputs.json', "r", encoding="utf-8") as file:
    notfinetuened_outputs = json.load(file)

# 비동기 호출을 위한 함수 정의
async def generate_project_data(index, prompt, model1, model2, retries=0):
    async with semaphore:  # 세마포어로 동시 호출 제한
        input_data = {"prompt": prompt, "model1": model1, "model2": model2}
        try:
            result = await chain.ainvoke(input_data)
            return index, result
        except Exception as e:
            print(f"❌ 오류 발생: {index} - {e}")
            if retries < MAX_RETRIES:
                print(f"🔄 재시도 {retries + 1}/{MAX_RETRIES} - {index}")
                return await generate_project_data(index, prompt, model1, model2, retries=retries + 1)
            else:
                print(f"⚠️ {index} 처리 실패 (최대 재시도 도달)")
                failed_requests.append({"index": index, "prompt": prompt, "model1": model1, "model2": model2})
                return index, None  # 실패한 요청은 None 반환

# None인 값만 다시 실행하는 메인 함수
async def retry_failed_tasks():
    progress_bar = tqdm(total=len(retry_indices), unit="task", disable=False)
    # model 1 = finetuned, model 2 = not finetuned
    tasks = [generate_project_data(i, dataset['test']['prompt'][i], finetuened_outputs[i], notfinetuened_outputs[i]) for i in retry_indices]
    completed_count = 0

    for future in asyncio.as_completed(tasks):
        index, result = await future
        results[index] = result
        completed_count += 1
        progress_bar.update(1)

        # 10개 단위로 진행률 출력 및 저장
        if completed_count % 10 == 0:
            progress_bar.n = completed_count
            # progress_bar.refresh()
            with open(OUTPUT_FILE, "w", encoding="utf-8") as json_file:
                json.dump(results, json_file, ensure_ascii=False, indent=4)

    progress_bar.close()

    # 최종 결과 저장
    with open(OUTPUT_FILE, "w", encoding="utf-8") as json_file:
        json.dump(results, json_file, ensure_ascii=False, indent=4)

    # 실패한 요청 저장
    if failed_requests:
        with open(FAILED_REQUESTS_FILE, "w", encoding="utf-8") as f:
            json.dump(failed_requests, f, ensure_ascii=False, indent=4)
        print(f"⚠️ {len(failed_requests)}개 요청 실패. {FAILED_REQUESTS_FILE}에 저장됨.")

    print("🎉 모든 작업 완료!")

# 비동기 실행
await retry_failed_tasks()


In [10]:
with open ("/Users/no.2/Desktop/GitHub/cpplab-sllm/data/eval_results/finetuned_notfinetuned_eval_results.json", "r", encoding="utf-8") as file:
    gpteval_results = json.load(file)

# 결과 확인
cnt_1 = 0
for i, result in enumerate(gpteval_results):
    if int(result['result']) == 1:
        cnt_1 += 1

print(f"파인튜닝 후 결과가 더 좋은 경우: {cnt_1}개")
print(f"파인튜닝 전 결과가 더 좋은 경우: {len(gpteval_results) - cnt_1}개")

print(f"파인튜닝 후 결과가 더 좋은 비율: {cnt_1 / len(gpteval_results) * 100:.2f}%")

파인튜닝 후 결과가 더 좋은 경우: 1227개
파인튜닝 전 결과가 더 좋은 경우: 573개
파인튜닝 후 결과가 더 좋은 비율: 68.17%


In [ ]:
# GPT 평가
# GPT vs fine-tuned

import asyncio
import json
import os
from tqdm.asyncio import tqdm  # 비동기 tqdm

# 저장된 데이터 경로
OUTPUT_FILE = "/Users/no.2/Desktop/GitHub/cpplab-sllm/data/eval_results/gpt_finetuned_eval_results.json"
FAILED_REQUESTS_FILE = "/Users/no.2/Desktop/GitHub/cpplab-sllm/data/eval_results/gpt_fintuned_eval_failed_requests.json"

from datasets import load_dataset
dataset = load_dataset("hyeongmin99/cpplab_sllm")

# 동시 작업 수 제한
semaphore = asyncio.Semaphore(15)
MAX_RETRIES = 3  # 최대 재시도 횟수

# 기존 저장된 데이터 불러오기
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
        try:
            results = json.load(f)
            print(f"✅ 기존 저장된 데이터 로드 성공 ({len(results)}개)")
        except json.JSONDecodeError:
            print("⚠️ 기존 데이터 로드 실패. 새로운 파일로 진행합니다.")
            results = [None] * len(dataset['test'])
else:
    results = [None] * len(dataset['test'])

# None인 데이터만 다시 실행할 리스트 생성
retry_indices = [i for i, result in enumerate(results) if result is None]
print(f"⚠️ {len(retry_indices)}개의 실패한 작업을 다시 시도합니다.")

# 실패한 요청 저장 리스트
failed_requests = []

with open('/Users/no.2/Desktop/GitHub/cpplab-sllm/data/inference_outputs/finetuned_outputs.json', "r", encoding="utf-8") as file:
    finetuened_outputs = json.load(file)
gpt_outputs = dataset['test']['output']

# 비동기 호출을 위한 함수 정의
async def generate_project_data(index, prompt, model1, model2, retries=0):
    async with semaphore:  # 세마포어로 동시 호출 제한
        input_data = {"prompt": prompt, "model1": model1, "model2": model2}
        try:
            result = await chain.ainvoke(input_data)
            return index, result
        except Exception as e:
            print(f"❌ 오류 발생: {index} - {e}")
            if retries < MAX_RETRIES:
                print(f"🔄 재시도 {retries + 1}/{MAX_RETRIES} - {index}")
                return await generate_project_data(index, prompt, model1, model2, retries=retries + 1)
            else:
                print(f"⚠️ {index} 처리 실패 (최대 재시도 도달)")
                failed_requests.append({"index": index, "prompt": prompt, "model1": model1, "model2": model2})
                return index, None  # 실패한 요청은 None 반환

# None인 값만 다시 실행하는 메인 함수
async def retry_failed_tasks():
    progress_bar = tqdm(total=len(retry_indices), unit="task", disable=False)
    # model 1 = finetuned, model 2 = gpt
    tasks = [generate_project_data(i, dataset['test']['prompt'][i], finetuened_outputs[i], gpt_outputs[i]) for i in retry_indices]
    completed_count = 0

    for future in asyncio.as_completed(tasks):
        index, result = await future
        results[index] = result
        completed_count += 1
        progress_bar.update(1)

        # 10개 단위로 진행률 출력 및 저장
        if completed_count % 10 == 0:
            progress_bar.n = completed_count
            # progress_bar.refresh()
            with open(OUTPUT_FILE, "w", encoding="utf-8") as json_file:
                json.dump(results, json_file, ensure_ascii=False, indent=4)

    progress_bar.close()

    # 최종 결과 저장
    with open(OUTPUT_FILE, "w", encoding="utf-8") as json_file:
        json.dump(results, json_file, ensure_ascii=False, indent=4)

    # 실패한 요청 저장
    if failed_requests:
        with open(FAILED_REQUESTS_FILE, "w", encoding="utf-8") as f:
            json.dump(failed_requests, f, ensure_ascii=False, indent=4)
        print(f"⚠️ {len(failed_requests)}개 요청 실패. {FAILED_REQUESTS_FILE}에 저장됨.")

    print("🎉 모든 작업 완료!")

# 비동기 실행
await retry_failed_tasks()

In [9]:
with open ("/Users/no.2/Desktop/GitHub/cpplab-sllm/data/eval_results/gpt_finetuned_eval_results.json", "r", encoding="utf-8") as file:
    gpteval_results = json.load(file)

# 결과 확인
cnt_1 = 0
for i, result in enumerate(gpteval_results):
    if int(result['result']) == 1:
        cnt_1 += 1

print(f"finetuned 모델이 더 좋은 결과를 보인 경우: {cnt_1}개")
print(f"gpt 모델이 더 좋은 결과를 보인 경우: {len(gpteval_results) - cnt_1}개")
# 비율
print(f"finetuned 모델이 더 좋은 비율: {cnt_1 / len(gpteval_results) * 100:.2f}%")

finetuned 모델이 더 좋은 결과를 보인 경우: 787개
gpt 모델이 더 좋은 결과를 보인 경우: 1013개
finetuned 모델이 더 좋은 비율: 43.72%


In [ ]:
# GPT 평가
# GPT vs not fine-tuned

import asyncio
import json
import os
from tqdm.asyncio import tqdm  # 비동기 tqdm

# 저장된 데이터 경로
OUTPUT_FILE = "/Users/no.2/Desktop/GitHub/cpplab-sllm/data/eval_results/gpt_notfinetuned_eval_results.json"
FAILED_REQUESTS_FILE = "/Users/no.2/Desktop/GitHub/cpplab-sllm/data/eval_results/gpt_notfintuned_eval_failed_requests.json"

from datasets import load_dataset
dataset = load_dataset("hyeongmin99/cpplab_sllm")

# 동시 작업 수 제한
semaphore = asyncio.Semaphore(15)
MAX_RETRIES = 3  # 최대 재시도 횟수

# 기존 저장된 데이터 불러오기
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
        try:
            results = json.load(f)
            print(f"✅ 기존 저장된 데이터 로드 성공 ({len(results)}개)")
        except json.JSONDecodeError:
            print("⚠️ 기존 데이터 로드 실패. 새로운 파일로 진행합니다.")
            results = [None] * len(dataset['test'])
else:
    results = [None] * len(dataset['test'])

# None인 데이터만 다시 실행할 리스트 생성
retry_indices = [i for i, result in enumerate(results) if result is None]
print(f"⚠️ {len(retry_indices)}개의 실패한 작업을 다시 시도합니다.")

# 실패한 요청 저장 리스트
failed_requests = []

with open('/Users/no.2/Desktop/GitHub/cpplab-sllm/data/inference_outputs/notfinetuned_outputs.json', "r", encoding="utf-8") as file:
    notfinetuened_outputs = json.load(file)
gpt_outputs = dataset['test']['output']

# 비동기 호출을 위한 함수 정의
async def generate_project_data(index, prompt, model1, model2, retries=0):
    async with semaphore:  # 세마포어로 동시 호출 제한
        input_data = {"prompt": prompt, "model1": model1, "model2": model2}
        try:
            result = await chain.ainvoke(input_data)
            return index, result
        except Exception as e:
            print(f"❌ 오류 발생: {index} - {e}")
            if retries < MAX_RETRIES:
                print(f"🔄 재시도 {retries + 1}/{MAX_RETRIES} - {index}")
                return await generate_project_data(index, prompt, model1, model2, retries=retries + 1)
            else:
                print(f"⚠️ {index} 처리 실패 (최대 재시도 도달)")
                failed_requests.append({"index": index, "prompt": prompt, "model1": model1, "model2": model2})
                return index, None  # 실패한 요청은 None 반환

# None인 값만 다시 실행하는 메인 함수
async def retry_failed_tasks():
    progress_bar = tqdm(total=len(retry_indices), unit="task", disable=False)
    # model 1 = finetuned, model 2 = gpt
    tasks = [generate_project_data(i, dataset['test']['prompt'][i], notfinetuened_outputs[i], gpt_outputs[i]) for i in retry_indices]
    completed_count = 0

    for future in asyncio.as_completed(tasks):
        index, result = await future
        results[index] = result
        completed_count += 1
        progress_bar.update(1)

        # 10개 단위로 진행률 출력 및 저장
        if completed_count % 10 == 0:
            progress_bar.n = completed_count
            # progress_bar.refresh()
            with open(OUTPUT_FILE, "w", encoding="utf-8") as json_file:
                json.dump(results, json_file, ensure_ascii=False, indent=4)

    progress_bar.close()

    # 최종 결과 저장
    with open(OUTPUT_FILE, "w", encoding="utf-8") as json_file:
        json.dump(results, json_file, ensure_ascii=False, indent=4)

    # 실패한 요청 저장
    if failed_requests:
        with open(FAILED_REQUESTS_FILE, "w", encoding="utf-8") as f:
            json.dump(failed_requests, f, ensure_ascii=False, indent=4)
        print(f"⚠️ {len(failed_requests)}개 요청 실패. {FAILED_REQUESTS_FILE}에 저장됨.")

    print("🎉 모든 작업 완료!")

# 비동기 실행
await retry_failed_tasks()


In [ ]:
with open ("/Users/no.2/Desktop/GitHub/cpplab-sllm/data/eval_results/gpt_notfinetuned_eval_results.json", "r", encoding="utf-8") as file:
    gpteval_results = json.load(file)

# 결과 확인
cnt_1 = 0
for i, result in enumerate(gpteval_results):
    if int(result['result']) == 1:
        cnt_1 += 1

print(f"notfinetuned 모델이 더 좋은 결과를 보인 경우: {cnt_1}개")
print(f"gpt 모델이 더 좋은 결과를 보인 경우: {len(gpteval_results) - cnt_1}개")

print(f"notfinetuned 모델이 더 좋은 비율: {cnt_1 / len(gpteval_results) * 100:.2f}%")

notfinetuned 모델이 더 좋은 결과를 보인 경우: 743개
gpt 모델이 더 좋은 결과를 보인 경우: 1057개
notfinetuned 모델이 더 좋은 비율: 41.28%
